In [ ]:
import src.imu_extraction as imu_extraction
import src.labeler as labeler
import src.model as modelfunc

In [2]:
label_mapping = None
model = None
unlabel_df = None
stored_sequences = None
settings = {
    "video_path": None,
    "imu_path": None,
    "from_scratch": True,
    "overlap": 0.50,
    "length": 10,
    "epochs": 5,
    "batch_size": 16,
    "dropout": 0.2,
    "dense_activation": "sigmoid",
    "LSTM_activation": "tanh",
    "LSTM_units": 256,
    "optimizer": "adam",
    "loss": "binary_crossentropy",
    "metrics": ["accuracy"],
    "learning_rate": 0.001,
    "weight_decay": None,
    "target_sequence_length": None
}

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Function to handle the selection of model type
def on_model_type_change(change):
    if change['new'] == 'Use Trained Model':
        zip_file_path_widget.layout.display = 'block'
    else:
        zip_file_path_widget.layout.display = 'none'

# Dropdown for selecting model type
model_type_widget = widgets.Dropdown(
    options=['Create New Model', 'Use Trained Model'],
    value='Create New Model',
    description='Model Type:'
)
model_type_widget.observe(on_model_type_change, names='value')

# Text input for zip file path
zip_file_path_widget = widgets.Text(
    value='',
    placeholder='Enter zip file path',
    description='Zip File Path:',
    layout=widgets.Layout(display='none')
)

# Function to update settings
def update_settings(change):
    settings[change.owner.description] = change.new

# Create widgets for each setting
setting_widgets = []
for key, value in settings.items():
    widget = widgets.Text(
        value=str(value),
        description=key,
        style={'description_width': 'initial'}
    )
    widget.observe(update_settings, names='value')
    setting_widgets.append(widget)

# Display widgets
display(model_type_widget, zip_file_path_widget)
for widget in setting_widgets:
    display(widget)

In [ ]:
print(settings)
print(settings['video_path'])

In [ ]:
if settings["imu_path"] == None:
    df = imu_extraction.extract_imu_data(settings["video_path"])
else:
    df = pd.read_csv(settings["imu_path"])
df = labeler.frame_index(settings["video_path"], df)
display(df)

In [ ]:
from IPython.display import Video, display, HTML
import ipywidgets as widgets
from IPython.display import display
import os

# Ensure the video path is correct
video_path = settings['video_path']

# Print the video path to verify it
print(f"Video path: {video_path}")

# Check if the file exists
if not os.path.exists(video_path):
    raise FileNotFoundError(f"Video file not found at path: {video_path}")

# Display the video
display(Video(video_path))

# Create input widgets
label_input = widgets.Text(description="Label:")
start_frame_input = widgets.IntText(description="Start Frame:")
end_frame_input = widgets.IntText(description="End Frame:")
add_button = widgets.Button(description="Add Row")
save_button = widgets.Button(description="Save List")

# List to store the inputs
annotations = []

# Function to add a row
def add_row(b):
    label = label_input.value
    start_frame = start_frame_input.value
    end_frame = end_frame_input.value
    annotations.append({"label": label, "start_frame": start_frame, "end_frame": end_frame})
    print(f"Added: {annotations[-1]}")

# Function to save the list
def save_list(b):
    print("Annotations saved:")
    for annotation in annotations:
        print(annotation)

# Attach functions to buttons
add_button.on_click(add_row)
save_button.on_click(save_list)

# Display the widgets
display(label_input, start_frame_input, end_frame_input, add_button, save_button)

In [ ]:
label_mapping = None
stored_sequences = None
results = modelfunc.run_model(blocks, settings,model=model df=df, label_mapping=label_mapping, stored_sequences=stored_sequences)
result_list, settings, model, prediction_df, label_mapping, df, padded_sequences, padded_labels = results
print(len(result_list), result_list)